In [259]:
# this notebook is sucessfully merging image1/image2 and image2/image3

In [260]:
import numpy as np
from PIL import Image
import cv2

In [261]:
folder = 'origin'
f1 = 'merged_stripped_002_005'
f2 = 'merged_stripped_002_007'

In [262]:
im1 =  cv2.imread(folder + '\\' + f1 + '.jpg')
im2 =  cv2.imread(folder + '\\' + f2 + '.jpg')

im1_gray = cv2.cvtColor(im1,cv2.COLOR_BGR2GRAY)
im2_gray = cv2.cvtColor(im2,cv2.COLOR_BGR2GRAY)

In [263]:
ht1, wd1 = im1_gray.shape
ht2, wd2= im2_gray.shape

In [264]:
################ main heights and widths #############################
print(ht1, wd1)
print(ht2, wd2)

3626 5667
3000 1841


In [265]:
# # works with im1/im2, im3, im4, im5, im6/im7
# trim_wd1 = int(wd1/2)
# trim_ht1 = int(ht1*0.6)

# trim_wd2 = int(wd2/2)
# trim_ht2 = int(ht2*0.6)

In [266]:
# 1st merge im1, im2, im3, im4, im5
# then merge im6, im7
# then use settings below to merge the two results
trim_wd1 = int(wd1*.2)
trim_ht1 = int(ht1*.2)

trim_wd2 = int(wd2*0.7)
trim_ht2 = int(ht2/4)

In [267]:
trimmed1 = im1_gray[:trim_ht1, wd1 - trim_wd1:]
trimmed2 = im2_gray[ht2-trim_ht2:, :trim_wd2]

In [268]:
# trimmed1 = im1[:trim_ht1, wd1 - trim_wd1:]
# trimmed2 = im2[ht2-trim_ht2:, :trim_wd2]

In [269]:
################ trimmed heights and widths #############################
print(trim_ht1, trim_wd1)
print(trim_ht2, trim_wd2)

725 1133
750 1288


In [270]:
# Image.fromarray(im1)

In [271]:
# Image.fromarray(trimmed1)

In [272]:
# Image.fromarray(trimmed2)

In [273]:
orb = cv2.ORB_create(50)

kp1, des1 = orb.detectAndCompute(trimmed1, None)
kp2, des2 = orb.detectAndCompute(trimmed2, None)

matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)

matches = matcher.match(des1, des2, None)

matches = sorted(matches, key = lambda x:x.distance)

In [274]:
# draw matches
imgX = cv2.drawMatches(trimmed1, kp1, trimmed2, kp2, matches[:1], None)

# uncomment to show matches
# cv2.imshow('matches',imgX)

cv2.waitKey(0)

-1

In [275]:
points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = kp1[match.queryIdx].pt
    points2[i, :] = kp2[match.trainIdx].pt

In [276]:
num = 0

# im1_doty = ht1 - trim_ht1 + points1[num][1]
im1_doty = ht1 - points1[num][1]
im1_dotx = wd1 - trim_wd1 + points1[num][0]

# im2_doty = points2[num][1]
im2_doty = trim_ht2 - points2[num][1]
im2_dotx = points2[num][0]

In [277]:
################ dot locs #############################
print(im1_doty, im1_dotx)
print(im2_doty, im2_dotx)

3136.0 5574.0
689.0 338.0


In [278]:
y_buf1 = int(round(im1_doty - im2_doty + ht2 - ht1))
x_buf1 = int(round(im1_dotx - im2_dotx + wd2 - wd1))

# y_buf1
# x_buf1

y_buf2 = int(round(im1_doty - im2_doty))
x_buf2 = int(round(im1_dotx - im2_dotx))

In [279]:
################ buffers #############################
print(y_buf1, x_buf1)
print(y_buf2, x_buf2)

1821 1410
2447 5236


In [280]:
# copyMakeBorder( src, dst, top, bottom, left, right, borderType, value )
im1_new = cv2.copyMakeBorder(im1, y_buf1, 0, 0, x_buf1, cv2.BORDER_CONSTANT)
# im2_new = cv2.copyMakeBorder(im2, 0, y_buf2, x_buf2, 0, cv2.BORDER_CONSTANT)

In [281]:
# convert colors: BGR to RGB
im1_new_rgb = cv2.cvtColor(im1_new, cv2.COLOR_BGR2RGB)
im2_rgb = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)

In [282]:
# use without scale
out1 = Image.fromarray(im1_new_rgb)
out2 = Image.fromarray(im2_rgb)

output = out1.copy()
output.paste(out2, (x_buf2, 0))

output.save(folder + '\\merged_' + f2 + '.jpg')

In [283]:
# # use for scale
# out1.putalpha(255)
# out2.putalpha(255)

# # makes the corner that interferes with scale transparent
# arr = np.array(out2)
# arr[-500:, :300, 3] = 0
# out2 = Image.fromarray(arr)

# output = out1.copy()
# output.paste(out2, (x_buf2, 0), out2)

# output = output.convert("RGB")

# output.save(folder + '\\merged_' + f2 + '.jpg')